In [3]:
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import cv2

# defining global variable path
image_path = "/home/jesterrexx/Documents/Dataset/CD data/training_set"

# ''function to load folder into arrays and 
# then it returns that same array'''
def loadImages(path):
    # Put files into lists and return them as one list of size 4
    image_files = sorted([os.path.join(path, 'train', file)
         for file in os.listdir(path + "/train") if      file.endswith('.jpg')])
 
    return image_files

In [8]:
try:

    import tensorflow as tf
    import cv2
    import os
    import pickle
    import numpy as np
    print("Library Loaded Successfully ..........")
except:
    print("Library not Found ! ")


class MasterImage(object):

    def __init__(self,PATH='', IMAGE_SIZE = 50):
        self.PATH = PATH
        self.IMAGE_SIZE = IMAGE_SIZE

        self.image_data = []
        self.x_data = []
        self.y_data = []
        self.CATEGORIES = []

        # This will get List of categories
        self.list_categories = []

    def get_categories(self):
        for path in os.listdir(self.PATH):
            if '.DS_Store' in path:
                pass
            else:
                self.list_categories.append(path)
        print("Found Categories ",self.list_categories,'\n')
        return self.list_categories

    def Process_Image(self):
        try:
            """
            Return Numpy array of image
            :return: X_Data, Y_Data
            """
            self.CATEGORIES = self.get_categories()
            for categories in self.CATEGORIES:                                                  # Iterate over categories

                train_folder_path = os.path.join(self.PATH, categories)                         # Folder Path
                class_index = self.CATEGORIES.index(categories)                                 # this will get index for classification

                for img in os.listdir(train_folder_path):                                       # This will iterate in the Folder
                    new_path = os.path.join(train_folder_path, img)                             # image Path

                    try:        # if any image is corrupted
                        image_data_temp = cv2.imread(new_path,cv2.IMREAD_GRAYSCALE)                 # Read Image as numbers
                        image_temp_resize = cv2.resize(image_data_temp,(self.IMAGE_SIZE,self.IMAGE_SIZE))
                        self.image_data.append([image_temp_resize,class_index])
                    except:
                        pass

            data = np.asanyarray(self.image_data)

            # Iterate over the Data
            for x in data:
                self.x_data.append(x[0])        # Get the X_Data
                self.y_data.append(x[1])        # get the label

            X_Data = np.asarray(self.x_data) / (255.0)      # Normalize Data
            Y_Data = np.asarray(self.y_data) 

            # reshape x_Data

            X_Data = X_Data.reshape(-1, self.IMAGE_SIZE, self.IMAGE_SIZE, 1)

            return X_Data, Y_Data
        except:
            print("Failed to run Function Process Image ")

    def pickle_image(self):

        """
        :return: None Creates a Pickle Object of DataSet
        """
        # Call the Function and Get the Data
        X_Data,Y_Data = self.Process_Image()

        # Write the Entire Data into a Pickle File
        pickle_out = open('X_Data','wb')
        pickle.dump(X_Data, pickle_out)
        pickle_out.close()

        # Write the Y Label Data
        pickle_out = open('Y_Data', 'wb')
        pickle.dump(Y_Data, pickle_out)
        pickle_out.close()

        print("Pickled Image Successfully ")
        return X_Data,Y_Data

    def load_dataset(self):

        try:
            # Read the Data from Pickle Object
            X_Temp = open('X_Data','rb')
            X_Data = pickle.load(X_Temp)

            Y_Temp = open('Y_Data','rb')
            Y_Data = pickle.load(Y_Temp)

            print('Reading Dataset from PIckle Object')

            return X_Data,Y_Data

        except:
            print('Could not Found Pickle File ')
            print('Loading File and Dataset  ..........')

            X_Data,Y_Data = self.pickle_image()
            return X_Data,Y_Data


if __name__ == "__main__":
    path = "/home/jesterrexx/Documents/Dataset/CD data/training_set"
    a = MasterImage(PATH=path,
                    IMAGE_SIZE=70)

    X_Data,Y_Data = a.load_dataset()
    print(X_Data.shape)

Library Loaded Successfully ..........
Reading Dataset from PIckle Object
(8005, 80, 80, 1)


In [11]:
print(X_Data.shape)
print(Y_Data.shape)

(8005, 80, 80, 1)
(8005,)


In [12]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X_Data, Y_Data, test_size=0.15, random_state=42)
number_of_train = x_train.shape[1]
number_of_test = x_test.shape[1]

In [13]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D,MaxPooling2D,Flatten,Dense,BatchNormalization,Dropout,MaxPool2D

In [25]:
model=Sequential([
    Conv2D(64,(3,3),activation='tanh',padding='same',input_shape=(80,80,1)),
    MaxPooling2D((2,2)),
    Dropout(0.2),
    BatchNormalization(),
    Conv2D(128,(3,3),activation='tanh',padding='same'),
    MaxPooling2D((2,2)),
    Dropout(0.25),
    BatchNormalization(),
    Conv2D(256,(3,3),activation='tanh',padding='same'),
    MaxPooling2D((2,2)),
    Dropout(0.3),
    BatchNormalization(),
    Flatten(),
    Dense(2,activation='sigmoid')
])

In [26]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 80, 80, 64)        640       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 40, 40, 64)        0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 40, 40, 64)        0         
_________________________________________________________________
batch_normalization_6 (Batch (None, 40, 40, 64)        256       
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 40, 40, 128)       73856     
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 20, 20, 128)       0         
_________________________________________________________________
dropout_7 (Dropout)          (None, 20, 20, 128)      

In [27]:
model.compile(optimizer='Adam',loss='mae',metrics=['acc'])

In [28]:
history=model.fit(x_train,y_train,epochs=15,validation_split=0.30,batch_size=50)

Epoch 1/15
96/96 [==============================] - 121s 1s/step - loss: 0.4133 - acc: 0.4490 - val_loss: 0.4995 - val_acc: 0.4995
Epoch 2/15
96/96 [==============================] - 117s 1s/step - loss: 0.3858 - acc: 0.4318 - val_loss: 0.4305 - val_acc: 0.4745
Epoch 3/15
96/96 [==============================] - 114s 1s/step - loss: 0.3788 - acc: 0.4681 - val_loss: 0.4953 - val_acc: 0.4775
Epoch 4/15
96/96 [==============================] - 114s 1s/step - loss: 0.3581 - acc: 0.4383 - val_loss: 0.4321 - val_acc: 0.4917
Epoch 5/15
96/96 [==============================] - 118s 1s/step - loss: 0.3585 - acc: 0.4368 - val_loss: 0.4873 - val_acc: 0.5690
Epoch 6/15
96/96 [==============================] - 119s 1s/step - loss: 0.3426 - acc: 0.4313 - val_loss: 0.4990 - val_acc: 0.5470
Epoch 7/15
96/96 [==============================] - 116s 1s/step - loss: 0.3524 - acc: 0.4332 - val_loss: 0.3910 - val_acc: 0.4623
Epoch 8/15
96/96 [==============================] - 114s 1s/step - loss: 0.3392 - a

In [30]:


history_1= model.fit(train_generator,
                       validation_data=test_generator,
                              epochs=30,
                              steps_per_epoch=125
                              )



NameError: name 'train_generator' is not defined

In [48]:
from tensorflow.keras.utils import to_categorical
y=to_categorical(y)
y.shape


(2188, 3)

In [49]:
y

array([[1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       ...,
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.]], dtype=float32)

In [51]:
print(x.shape)
x

AttributeError: 'list' object has no attribute 'shape'

In [8]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,random_state=0,test_size=0.2)

In [9]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D,MaxPooling2D,Flatten,Dense,BatchNormalization,Dropout,MaxPool2D

In [32]:
model=Sequential([
    Conv2D(64,(3,3),activation='relu',padding='same',input_shape=(100,100,1)),
    MaxPooling2D((2,2)),
    Dropout(0.2),
    BatchNormalization(),
    Conv2D(128,(3,3),activation='relu',padding='same'),
    MaxPooling2D((2,2)),
    Dropout(0.25),
    BatchNormalization(),
    Conv2D(256,(3,3),activation='relu',padding='same'),
    MaxPooling2D((2,2)),
    Dropout(0.3),
    BatchNormalization(),
    Flatten(),
    Dense(2,activation='sigmoid')
])

In [33]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 100, 100, 64)      640       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 50, 50, 64)        0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 50, 50, 64)        0         
_________________________________________________________________
batch_normalization_6 (Batch (None, 50, 50, 64)        256       
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 50, 50, 128)       73856     
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 25, 25, 128)       0         
_________________________________________________________________
dropout_7 (Dropout)          (None, 25, 25, 128)      

In [34]:
model.compile(optimizer='Adam',loss='mae',metrics=['acc'])

In [32]:
history=model.fit(x_train,y_train,epochs=1,validation_split=0.30,batch_size=50)

96/96 [==============================] - 108s 1s/step - loss: 0.2714 - acc: 0.4000 - val_loss: 0.4135 - val_acc: 0.4731
